<a href="https://colab.research.google.com/github/BarbJS/Gerenciamento-de-biblioteca-digital/blob/main/gestao_arquivos_digitais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *Projeto: Sistema de gerenciamento de biblioteca digital*

In [ ]:
# -*- coding: utf-8 -*-
# Importando as bibliotecas necessárias
import os  # Biblioteca para interagir com o sistema operacional (criar pastas, listar arquivos)
import shutil  # Biblioteca para operações de alto nível com arquivos (mover arquivos)
import re  # Biblioteca para trabalhar com expressões regulares (usaremos para achar o ano no nome do arquivo)
import datetime  # Biblioteca para trabalhar com datas e horas (usaremos para pegar o ano atual)